# Quickstart Guide for Strands Agents / Strands Agents 快速入门指南

Strands Agents is a powerful framework for building AI agents that can interact with AWS services and perform complex tasks. This quickstart guide will help you get started with creating your first Strands agent.

Strands Agents 是一个强大的框架，用于构建可以与 AWS 服务交互并执行复杂任务的 AI 代理。本快速入门指南将帮助您开始创建您的第一个 Strands 代理。




## Creating Your First Agent / 创建您的第一个代理

Lets get an overview of the agentic components needed.

让我们了解一下所需的代理组件。

### 1. Create a simple Agent / 创建一个简单的代理

This will create an agent with the default model provider, [Amazon Bedrock](https://aws.amazon.com/bedrock/), and the default model, Claude 3.7 Sonnet, in the region of your AWS setup. While the agent runs in the same local environment as it is being invoked, Amazon Bedrock models will run in an AWS account and your agent will invoke the model in the cloud account. The architecture looks as following:

这将创建一个使用默认模型提供商 [Amazon Bedrock](https://aws.amazon.com/bedrock/) 和默认模型 Claude 3.7 Sonnet 的代理，位于您的 AWS 设置区域中。虽然代理在被调用的同一本地环境中运行，但 Amazon Bedrock 模型将在 AWS 账户中运行，您的代理将调用云账户中的模型。架构如下所示：
<center>
    <img src="./images/simple_agent.png" width="75%" />
</center>



In [11]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import warnings
warnings.filterwarnings(action="ignore", message=r"datetime.datetime.utcnow") 

from strands import Agent
# Initialize your agent
agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    system_prompt="You are a helpful assistant that provides concise responses."
)

# Send a message to the agent
response = agent("Hello! Tell me a joke in Chinese.")

你知道为什么鱼不能玩电脑吗？因为它们害怕网络钓鱼！

(Translation: Do you know why fish can't use computers? Because they're afraid of phishing!)

### 2. Add Tools to the Agent / 为代理添加工具

The [strands-agents-tools](https://github.com/strands-agents/tools) repository provides some in-built tools which you can import. You can also create custom tools using the `@tool` decorator. We can create agents with built-in and custom tools. For instance, adding the built-in tool of a calculator and a custom tool for getting the weather you get the following architecture:

[strands-agents-tools](https://github.com/strands-agents/tools) 仓库提供了一些可以导入的内置工具。您也可以使用 `@tool` 装饰器创建自定义工具。我们可以创建具有内置和自定义工具的代理。例如，添加计算器的内置工具和获取天气的自定义工具，您将得到以下架构：
<center>
    <img src="./images/agent_with_tools.png" width="75%" />
</center>
Implementing this architecture you have the following:

实现此架构，您需要以下代码：

In [3]:
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather.")

response = agent("What is the weather today? Answer in Chinese.")
print(response)

我需要为您查询今天的天气。请稍等，我将使用天气工具来获取这些信息。
Tool #1: weather
根据查询结果，今天的天气是晴朗的（sunny）。今天是一个阳光明媚的好日子！您可以安排户外活动，不必担心下雨的问题。根据查询结果，今天的天气是晴朗的（sunny）。今天是一个阳光明媚的好日子！您可以安排户外活动，不必担心下雨的问题。




### 4. Model Provider / 模型提供商

The default model provider is [Amazon Bedrock](https://aws.amazon.com/bedrock/) and the default model is Claude 3.7 Sonnet in the region of your current AWS environment. You can specify a different model in Amazon Bedrock providing the model ID string directly:

默认的模型提供商是 [Amazon Bedrock](https://aws.amazon.com/bedrock/)，默认模型是您当前 AWS 环境区域中的 Claude 3.7 Sonnet。您可以通过直接提供模型 ID 字符串在 Amazon Bedrock 中指定不同的模型：

In [4]:
from strands import Agent

agent = Agent(model="anthropic.claude-3-5-haiku-20241022-v1:0")
print(agent.model.config)

{'model_id': 'anthropic.claude-3-5-haiku-20241022-v1:0'}



For more control over the model configuration, you can create a `BedrockModel` provider instance:

为了更好地控制模型配置，您可以创建一个 `BedrockModel` 提供商实例：

In [5]:
import boto3
from strands import Agent
from strands.models import BedrockModel

# Create a BedrockModel
bedrock_model = BedrockModel(
    model_id="anthropic.claude-3-5-haiku-20241022-v1:0",
    region_name='us-west-2',
    temperature=0.3,
)

agent = Agent(model=bedrock_model)
print(agent.model.config)

{'model_id': 'anthropic.claude-3-5-haiku-20241022-v1:0', 'temperature': 0.3}



More details on the available model providers on the [Model Provider Quickstart page](https://strandsagents.com/0.1.x/user-guide/quickstart/#model-providers)

有关可用模型提供商的更多详细信息，请参阅[模型提供商快速入门页面](https://strandsagents.com/0.1.x/user-guide/quickstart/#model-providers)


**Congratulations !! Now you have learned how to build a simple agent using Strands!!**

**恭喜！！现在您已经学会了如何使用 Strands 构建简单的代理！！**

## Lets Build a Task-Specific Agent - RecipeBot 🍽️ 

##  让我们构建一个特定任务的agent -  食谱机器人🍽️
Lets create a practical example of a task-specific agent. We create a `RecipeBot` that recommends recipes and answers any cooking related questions. Lets dive in !!

让我们创建一个特定任务代理的实际示例。我们创建一个 `RecipeBot`，它推荐食谱并回答任何与烹饪相关的问题。让我们深入了解！！

Here's what we will create :

这是我们将要创建的内容：

<center>
    <img src="./images/interactive_recipe_agent.png" width="75%" />
</center>

In [6]:
from strands import Agent, tool
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException
import logging

# Configure logging
logging.getLogger("strands").setLevel(logging.INFO)

# Define a websearch tool
@tool
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"


In [7]:
# Create a recipe assistant agent
recipe_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    system_prompt="""You are RecipeBot, a helpful cooking assistant.
    Help users find recipes based on ingredients and answer cooking questions.
    Use the websearch tool to find recipes when users mention ingredients or to
    look up cooking information. And finally give the respose in Chinese""",
    # Import the websearch tool we created above
    tools=[websearch],
)

In [8]:
response = recipe_agent("Suggest a recipe with chicken and broccoli.")

# Other examples:
# response = recipe_agent("How do I cook quinoa?")
# response = recipe_agent("How can I substitute white wine in shrimp pasta?")
# response = recipe_agent("What are the health benefits of asparagus?")
print(response)

我来帮您寻找鸡肉和西兰花的食谱。让我搜索一些适合的菜谱。
Tool #1: websearch
根据搜索结果，我为您找到了几道美味的鸡肉和西兰花食谱。以下是一个简单易做的选择：

# 15分钟快手鸡肉西兰花

## 材料：
- 鸡胸肉，切成小块
- 西兰花，切成小朵
- 2-3瓣大蒜，切碎
- 1茶匙姜末
- 酱汁材料：
  - 3汤匙酱油
  - 1汤匙蜂蜜或红糖
  - 1汤匙醋
  - 1茶匙芝麻油
  - 适量清水
  - 1茶匙玉米淀粉调匀

## 做法：
1. 将鸡肉切成均匀的小块，用少量盐、胡椒和1茶匙玉米淀粉腌制几分钟。
2. 将所有酱汁材料混合均匀，备用。
3. 热锅加油，中高火将鸡肉煸炒至表面变色（约2-3分钟）。
4. 加入西兰花，继续翻炒1-2分钟。
5. 加入蒜末和姜末，炒香。
6. 倒入调好的酱汁，盖上锅盖焖约2-3分钟，让西兰花变得脆嫩。
7. 打开锅盖，继续翻炒直到酱汁变稠并均匀裹在鸡肉和西兰花上。
8. 出锅前可撒上少量芝麻点缀。

## 建议搭配：
可以搭配白米饭、糙米或面条一起食用，制作一顿营养均衡的美味晚餐。

这道菜只需要15分钟就能完成，比外卖更健康美味。酱汁可以根据个人口味调整甜度和咸度。西兰花保持脆嫩，鸡肉嫩滑多汁，是一道非常适合快节奏生活的健康营养餐。

希望您喜欢这个食谱！如果您想尝试其他风格的鸡肉西兰花食谱，我可以为您推荐焗烤芝士鸡肉西兰花饭或是中式蚝油鸡肉西兰花。根据搜索结果，我为您找到了几道美味的鸡肉和西兰花食谱。以下是一个简单易做的选择：

# 15分钟快手鸡肉西兰花

## 材料：
- 鸡胸肉，切成小块
- 西兰花，切成小朵
- 2-3瓣大蒜，切碎
- 1茶匙姜末
- 酱汁材料：
  - 3汤匙酱油
  - 1汤匙蜂蜜或红糖
  - 1汤匙醋
  - 1茶匙芝麻油
  - 适量清水
  - 1茶匙玉米淀粉调匀

## 做法：
1. 将鸡肉切成均匀的小块，用少量盐、胡椒和1茶匙玉米淀粉腌制几分钟。
2. 将所有酱汁材料混合均匀，备用。
3. 热锅加油，中高火将鸡肉煸炒至表面变色（约2-3分钟）。
4. 加入西兰花，继续翻炒1-2分钟。
5. 加入蒜末和姜末，炒香。
6. 倒入调好的酱汁，盖上锅盖焖约2-3分钟，让西兰花变得脆嫩。
7. 打开锅盖，继续翻炒直到酱汁变稠并均匀裹在鸡肉和西兰花上。
8. 出锅前可

#### And that's it! We now have a running usecase agent with tools in just a few lines of code 🥳.
#### 就是这样！我们现在只用几行代码就有了一个带工具的运行用例代理 🥳。

